In [1]:
!pip3 install tiktoken


In [2]:
import importlib
import tiktoken


In [3]:
tokenizer=tiktoken.get_encoding("gpt2")

In [5]:
with open("Shikhar_Goel_Resume.txt","r",encoding="utf-8") as f:
    raw_text=f.read()
encoded_text=tokenizer.encode(raw_text)
print(len(raw_text))

1547


In [6]:
encoded_text

[2484,
 1134,
 9869,
 1514,
 417,
 198,
 198,
 47,
 2433,
 45429,
 73,
 11,
 3794,
 28714,
 8298,
 198,
 43916,
 2154,
 5774,
 21599,
 198,
 14542,
 70,
 21,
 21599,
 31,
 14816,
 13,
 785,
 198,
 198,
 49138,
 21293,
 198,
 198,
 17309,
 16090,
 13613,
 6011,
 636,
 12,
 2435,
 670,
 1998,
 290,
 7667,
 3725,
 286,
 4755,
 2426,
 2300,
 13,
 3395,
 291,
 6985,
 290,
 3703,
 12,
 17107,
 351,
 922,
 40118,
 11,
 26564,
 4678,
 13,
 198,
 198,
 15739,
 2171,
 198,
 198,
 15167,
 2229,
 3303,
 1058,
 269,
 11,
 66,
 4880,
 11,
 29412,
 198,
 198,
 37,
 28216,
 198,
 198,
 6601,
 8573,
 290,
 978,
 7727,
 907,
 198,
 198,
 7293,
 316,
 298,
 198,
 198,
 37573,
 4673,
 29251,
 588,
 11192,
 273,
 11125,
 198,
 198,
 7293,
 316,
 298,
 198,
 198,
 31391,
 25,
 36216,
 15813,
 2193,
 290,
 584,
 12782,
 198,
 198,
 37,
 28216,
 198,
 198,
 35,
 12721,
 198,
 198,
 7293,
 316,
 298,
 198,
 198,
 19314,
 7582,
 36883,
 198,
 198,
 37,
 28216,
 198,
 198,
 26416,
 27828,
 198,
 198,
 37,
 28216

In [10]:
context_size=4
x=encoded_text[:context_size]
y=encoded_text[1:context_size+1]

[2484, 1134, 9869, 1514]


In [14]:
for i in range (1,context_size+1):
    input_llm=encoded_text[:i]
    target_llm=encoded_text[i]
    print(input_llm,"-->",target_llm)
    print(tokenizer.decode(input_llm),"-->",tokenizer.decode([target_llm]))

[2484] --> 1134
Sh --> ik
[2484, 1134] --> 9869
Shik --> har
[2484, 1134, 9869] --> 1514
Shikhar -->  Go
[2484, 1134, 9869, 1514] --> 417
Shikhar Go --> el


# data loader


In [23]:
from torch.utils.data import Dataset,DataLoader
class GPTdataset(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]
        
        ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
        
        for i in range(0,len(ids)-max_length,stride):
            input_chunk=ids[i:i+max_length]
            target_chunk=ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
        
                 

In [24]:
def gpt_data_loader(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
   
    tokenizer = tiktoken.get_encoding("gpt2")
    

    dataset = GPTdataset(txt, tokenizer, max_length, stride)
    
   
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return data_loader
    

In [28]:
import torch
data_loader=gpt_data_loader(raw_text,batch_size=4,max_length=4,stride=4,shuffle=False)
data_iter=iter(data_loader)
next(data_iter)

[tensor([[ 2484,  1134,  9869,  1514],
         [  417,   198,   198,    47],
         [ 2433, 45429,    73,    11],
         [ 3794, 28714,  8298,   198]]),
 tensor([[ 1134,  9869,  1514,   417],
         [  198,   198,    47,  2433],
         [45429,    73,    11,  3794],
         [28714,  8298,   198, 43916]])]